In [1]:
# Add custom import path

import sys
sys.path.insert(0, '/home/jacobsuwang/Documents/UTA2018/NEURAL-NETS/ATTENTION/CODE/01-import-folder')

In [44]:
import utils
import random
import numpy as np

VOCAB = set(['PAD','EOS','1','2','3','4','5','6','7','8','9','0'])
NUMBERS = ['1','2','3','4','5','6','7','8','9','0']
MAX_LEN = len(NUMBERS) + 2
WORD2IDX = {'PAD':0,'EOS':1,'1':2,'2':3,'3':4,'4':5,'5':6,'6':7,'7':8,'8':9,'9':10,'0':11}
IDX2WORD = {idx:word for word,idx in WORD2IDX.iteritems()}
BATCH_SIZE = 10


def random_datum(n):
    input_seq = list(np.random.choice(NUMBERS, n, replace=False)) 
        # e.g. ['5', '6', '3', '9', '1'].
    sorted_seq = sorted(input_seq)
    output_seq = [input_seq.index(word)+2 for word in sorted_seq]
        # index in ascending.
        # e.g. [4, 2, 0, 1, 3], for the input above.
    input_seq = input_seq
    return input_seq, output_seq

def encode_seq(seq):
    return [WORD2IDX[word] for word in seq]

def decode_seq(seq):
    return [IDX2WORD[idx] for idx in seq]

def decode_order(seq):
    return [aug_idx-2 for aug_idx in seq]

def decode_by_index(seq, idx_seq, end_idx, correction=-2): # -2: PAD and EOS
    decoded = []
    for idx in range(end_idx):
        decoded.append(seq[idx_seq[idx]+correction])
    return decoded

def random_batch(batch_size, input_length_from=2, input_length_to=MAX_LEN-2):
    if input_length_from >= input_length_to:
        raise ValueError('length_from >= length_to')
    input_lengths = np.random.randint(input_length_from, input_length_to, size=batch_size)
    input_batch, output_batch = [], []
    for length in input_lengths:
        input_seq, output_seq = random_datum(length)
        input_batch.append(encode_seq(input_seq))
        output_batch.append(output_seq)
    return input_batch, output_batch    


In [16]:
a,b = random_batch(3)
print a
print b

[[5, 10], [4, 3, 5, 9, 2, 10], [4, 11, 8, 2, 10, 7]]
[[2, 3], [6, 3, 2, 4, 5, 7], [3, 5, 2, 7, 4, 6]]


In [19]:
decode_by_index(a[0], b[0], 2)

[5, 10]

In [20]:
utils.batch(a)

(array([[ 5,  4,  4],
        [10,  3, 11],
        [ 0,  5,  8],
        [ 0,  9,  2],
        [ 0,  2, 10],
        [ 0, 10,  7]], dtype=int32), [2, 6, 6])

In [21]:
utils.batch(b)

(array([[2, 6, 3],
        [3, 3, 5],
        [0, 2, 2],
        [0, 4, 7],
        [0, 5, 4],
        [0, 7, 6]], dtype=int32), [2, 6, 6])

In [ ]:
'''
Comments:

- Decoder alone performs quite horribly. Not learning anything almost.

'''

In [4]:
import numpy as np
import tensorflow as tf

from tensorflow.contrib.rnn import LSTMCell, LSTMStateTuple

In [62]:
# Graph

tf.reset_default_graph()
sess = tf.InteractiveSession()

vocab_size = len(VOCAB)
embedding_size = 20
hidden_size = 15

inputs = tf.placeholder(shape=(None,None), dtype=tf.int32, name='inputs')
inputs_length = tf.placeholder(shape=(None,), dtype=tf.int32, name='inputs_length') 
targets = tf.placeholder(shape=(None,None), dtype=tf.int32, name='targets')
# targets_length = tf.placeholder(shape=(None,), dtype=tf.int32, name='targets_length')

embeddings = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0), dtype=tf.float32)
inputs_embedded = tf.nn.embedding_lookup(embeddings, inputs) # [max_time,batch_size,emb_size]

cell = LSTMCell(hidden_size)
max_time, batch_size = tf.unstack(tf.shape(inputs))

W = tf.Variable(tf.random_uniform([hidden_size, vocab_size], -1, 1), dtype=tf.float32)
b = tf.Variable(tf.zeros([vocab_size]), dtype=tf.float32)

eos_time_slice = tf.ones([batch_size], dtype=tf.int32, name='EOS') 
pad_time_slice = tf.zeros([batch_size], dtype=tf.int32, name='PAD')
eos_step_embedded = tf.nn.embedding_lookup(embeddings, eos_time_slice) 
pad_step_embedded = tf.nn.embedding_lookup(embeddings, pad_time_slice)

out_length = inputs_length + 3

def loop_fn_initial():
#     initial_elements_finished = (0 >= decoder_lengths) 
    initial_elements_finished = (0 >= out_length)
    initial_input = eos_step_embedded                 
#     initial_cell_state = encoder_final_state
    initial_cell_state = cell.zero_state(batch_size, dtype=tf.float32) # (c:[?,15],h:[?,15])
    initial_cell_output = None 
    initial_loop_state = None 
    return (initial_elements_finished,
            initial_input,
            initial_cell_state,
            initial_cell_output,
            initial_loop_state)

def loop_fn_transition(time, previous_output, previous_state, previous_loop_state):
    def get_next_input(): 
        
#         print 'prev_out: ', previous_output
#         print 'prev_state: ', previous_state
#         assert 1==0
#         prev_out:  Tensor("rnn/while/lstm_cell/mul_2:0", shape=(?, 15), dtype=float32)
#         prev_state:  LSTMStateTuple(c=<tf.Tensor 'rnn/while/lstm_cell/add_1:0' shape=(?, 15) dtype=float32>, 
#                                     h=<tf.Tensor 'rnn/while/lstm_cell/mul_2:0' shape=(?, 15) dtype=float32>)
        
        output_logits = tf.add(tf.matmul(previous_output, W), b)
        prediction = tf.argmax(output_logits, axis=1)
        next_input = tf.nn.embedding_lookup(embeddings, prediction)
        return next_input
#     elements_finished = (time >= decoder_lengths) 
    elements_finished = (time >= out_length) 
    finished = tf.reduce_all(elements_finished) 
    inpt = tf.cond(finished, lambda: pad_step_embedded, get_next_input)
    state = previous_state
    output = previous_output
    loop_state = None
    return (elements_finished,
            inpt, 
            state,
            output,
            loop_state)

def loop_fn(time, previous_output, previous_state, previous_loop_state):
    if previous_state is None:
        assert previous_output is None and previous_state is None
        return loop_fn_initial()
    else:
        return loop_fn_transition(time, previous_output, previous_state, previous_loop_state)
    
outputs_ta, final_state, _ = tf.nn.raw_rnn(cell, loop_fn)
outputs = outputs_ta.stack()

out_max_step, out_batch_size, out_dim = tf.unstack(tf.shape(outputs))
outputs_flat = tf.reshape(outputs, (-1, out_dim)) # [max_time, batch_size*emb_size]?
logits_flat = tf.add(tf.matmul(outputs_flat, W), b)
logits = tf.reshape(logits_flat, (out_max_step, out_batch_size, vocab_size))
prediction = tf.cast(tf.argmax(logits, 2), dtype=tf.int32)

# [SU] report accuracy here
correct_raw = tf.cast(tf.equal(prediction, targets), tf.int32)
mask = tf.cast(tf.not_equal(targets, WORD2IDX['PAD']), tf.int32)
total_seqlen = tf.cast(tf.reduce_sum(inputs_length), tf.float32)
correct = tf.multiply(correct_raw, mask)
accuracy = tf.cast(tf.reduce_sum(correct)-BATCH_SIZE, tf.float32) / total_seqlen

stepwise_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
    labels=tf.one_hot(targets, depth=vocab_size, dtype=tf.float32),
    logits=logits
)
loss = tf.reduce_mean(stepwise_cross_entropy)
train_op = tf.train.AdamOptimizer().minimize(loss)

init = tf.global_variables_initializer()
sess.run(init)

In [63]:
inputs_embedded

<tf.Tensor 'embedding_lookup:0' shape=(?, ?, 20) dtype=float32>

In [ ]:
#         prev_out:  Tensor("rnn/while/lstm_cell/mul_2:0", shape=(?, 15), dtype=float32)
#         prev_state:  LSTMStateTuple(c=<tf.Tensor 'rnn/while/lstm_cell/add_1:0' shape=(?, 15) dtype=float32>, 
#                                     h=<tf.Tensor 'rnn/while/lstm_cell/mul_2:0' shape=(?, 15) dtype=float32>)

In [ ]:
'''
NOTES:

- LSTM cell looks at 1 time-step at a time, so c/h: [batch_size=?, emb_size=15] 
- * I got use current time-step (i.e. time \in [0, max_time) ) to slice [time, batch_size=?, emb_size=15]
  from inputs_embedded, which has dim [max_time, batch_size, emb_size].
  This should result in a [batch_size, emb_size] memory matrix (at some time/word t).
- The above is not right, because for each sentence, I want to attend to a memory matrix of
  the shape [max_time, emb_size] [<hello>, <world>, <!>], for instance. 
  Which means my attention should be over the entire inputs_embedded tensor, and normalize properly.

'''

In [61]:
def next_feed(batch_size):
    batch_inputs, batch_outputs = random_batch(batch_size)
    inputs_, inputs_length_ = utils.batch(batch_inputs)
    targets_, _ = utils.batch([seq + [WORD2IDX['EOS']] + [WORD2IDX['PAD']]*2 for seq in batch_outputs])
    return {
        inputs: inputs_,
        inputs_length: inputs_length_,
        targets: targets_
    }

loss_track = []

# max_batches = 3001
num_epoches = 10
batches_in_epoch = 1000
num_test_batches = 100

try:
    for e in range(num_epoches):
        print 'Epoch: {}'.format(e+1)
        print
        for batch in range(batches_in_epoch):
            fd = next_feed(BATCH_SIZE)
            _, l = sess.run([train_op, loss], fd)
            loss_track.append(l)
#         print('Batch {}'.format(batch))
        print('  minibatch loss: {} | accuracy: {}'.format(*sess.run([loss, accuracy], fd)))
        predict_, lengths_ = sess.run([prediction, inputs_length], fd)
        for i, (inp, pred, tar, length) in enumerate(zip(fd[inputs].T, predict_.T, fd[targets].T, lengths_)):
            print('  sample {}:'.format(i + 1))
            decoded_inp = decode_seq(inp)
            print('    input     > {}'.format(decoded_inp))
            print('    predicted > {}'.format(decode_by_index(decoded_inp, pred, end_idx=length)))
            print('    target    > {}'.format(decode_by_index(decoded_inp, tar, end_idx=length)))
            if i >= 2:
                break
        print

    # EVALUATE ON A BIG TEST SET
    loss_track, accuracy_track = [], []
    for _ in range(num_test_batches):
        fd = next_feed(BATCH_SIZE)
        l, a = sess.run([loss, accuracy], fd)
        loss_track.append(l)
        accuracy_track.append(a)
    print('Evaluation results (on {} batches):'.format(num_test_batches))
    print('  average loss: {} | average accuracy {}'.format(np.mean(loss_track), np.mean(accuracy_track)))
    print
    
except KeyboardInterrupt:
    print('training interrupted')            

Epoch: 1



ValueError: Fetch argument <tf.Operation 'Adam' type=NoOp> cannot be interpreted as a Tensor. (Operation name: "Adam"
op: "NoOp"
input: "^Adam/update_Variable/group_deps"
input: "^Adam/update_Variable_1/ApplyAdam"
input: "^Adam/update_Variable_2/ApplyAdam"
input: "^Adam/update_rnn/lstm_cell/weights/ApplyAdam"
input: "^Adam/update_rnn/lstm_cell/biases/ApplyAdam"
input: "^Adam/Assign"
input: "^Adam/Assign_1"
 is not an element of this graph.)

In [43]:
next_feed(5)

{<tf.Tensor 'inputs:0' shape=(?, ?) dtype=int32>: array([[ 5,  3, 11,  5, 10],
        [11, 11,  5,  9,  8],
        [10,  5,  6,  4,  6],
        [ 3,  4,  0,  6,  0],
        [ 7,  9,  0,  7,  0],
        [ 2,  2,  0,  3,  0],
        [ 4,  8,  0,  2,  0],
        [ 8,  7,  0, 11,  0],
        [ 6, 10,  0,  0,  0]], dtype=int32),
 <tf.Tensor 'inputs_length:0' shape=(?,) dtype=int32>: [9, 9, 3, 8, 3],
 <tf.Tensor 'targets:0' shape=(?, ?) dtype=int32>: array([[ 3,  3,  2,  9,  4],
        [ 7,  7,  3,  8,  3],
        [ 5,  2,  4,  7,  2],
        [ 8,  5,  1,  4,  1],
        [ 2,  4,  0,  2,  0],
        [10,  9,  0,  5,  0],
        [ 6,  8,  0,  6,  0],
        [ 9,  6,  0,  3,  0],
        [ 4, 10,  0,  1,  0],
        [ 1,  1,  0,  0,  0],
        [ 0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0]], dtype=int32)}

In [34]:
outputs

<tf.Tensor 'TensorArrayStack/TensorArrayGatherV3:0' shape=(?, ?, 15) dtype=float32>

In [36]:
outputs_flat

<tf.Tensor 'Reshape:0' shape=(?, ?) dtype=float32>

In [37]:
out_dim

<tf.Tensor 'unstack_1:2' shape=() dtype=int32>

In [31]:
cell.zero_state(batch_size=batch_size, dtype=tf.float32)

LSTMStateTuple(c=<tf.Tensor 'zeros_1:0' shape=(?, 15) dtype=float32>, h=<tf.Tensor 'zeros_2:0' shape=(?, 15) dtype=float32>)

In [30]:
batch_size

<tf.Tensor 'unstack:1' shape=() dtype=int32>

In [13]:
inputs_embedded

<tf.Tensor 'embedding_lookup:0' shape=(?, ?, 20) dtype=float32>

In [23]:
max_time

<tf.Tensor 'unstack:0' shape=() dtype=int32>